In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
import importlib
import mr_rrn, data_aux
from tensorflow.contrib import learn

In [ ]:
# Settings
#examples=50000
num_steps = 20
num_seq = 6
n_hidden_enc = 300
n_hidden_con = 300
n_hidden_dec = 300
batch_size = 128
learning_rate = 0.001
#embedding_shape = [3, 3] # [voc_size, emb_dim]

In [ ]:
# Load data
importlib.reload(mr_rrn)
#X = data_aux.generate_test_data_sequence(examples=examples, num_seq=num_seq, num_steps=num_steps)
#x_data, y_data = X[:,:-1,:], X[:,1:,:]
data_sequences, data_len, vocab_processor = data_aux.generate_data_ubuntu(max_utterances = num_seq, max_tokens = num_steps)
batches = data_aux.batch_iter([data_sequences, data_len],batch_size=batch_size, num_epochs=5)

# Build graph
timer = datetime.now()
tf.reset_default_graph()
(vocab_input, sequence_input, length_input, total_loss, train_step) = mr_rrn.build_graph([len(vocab_processor.vocabulary_), 128], num_seq, num_steps, batch_size, n_hidden_enc, n_hidden_con, n_hidden_dec, learning_rate, gradient_clipping = 0.)
timer = datetime.now() - timer
print('Time build graph: ', timer)

# Train
W_embedding = data_aux.random_embedding(vocab_processor)
#W_embedding = np.eye(3)
acc_loss = 0
with tf.Session() as sess:
    timer = datetime.now()
    sess.run(tf.initialize_all_variables(), feed_dict={vocab_input : W_embedding})
    timer = datetime.now() - timer
    print('Time init graph: ', timer)
    timer = datetime.now()
    for i, batch in enumerate(batches):
        seq_batch, len_batch = batch
        feed_dict = {sequence_input : seq_batch,
                     length_input : len_batch
                     }
        loss, _ = sess.run([total_loss, train_step], feed_dict=feed_dict)
        print(loss)
        if i > 5:
            break
timer = datetime.now() - timer
print('Time run graph: ', timer)